In [ ]:
# default_exp utils.ann_retrieval

## ANN Retrieval
> Implementation of ANN (Approximate Nearest Neighbor) based retrieving algorithms.

A typical recommender system has two phases: preference learning and recommendation retrieval. While the former can be done offline, the latter needs to be fast. However, the cost of linearly scanning through the whole set of billions of items are prohibitive or sometimes even impossible.

To compute the nearest neighbors in the embedding space, the system can exhaustively score every potential candidate. Exhaustive scoring can be expensive for very large corpora, but you can use either of the following strategies to make it more efficient:
- If the query embedding is known statically, the system can perform exhaustive scoring offline, precomputing and storing a list of the top candidates for each query. This is a common practice for related-item recommendation.
- Use approximate nearest neighbors.

Here, we are using the 2nd option - The ANN.

<img src='https://github.com/recohut/reco-static/raw/master/media/diagrams/ann_retrieval.svg'>

As shown in the above diagram, there are two main steps in the embedding-based retrieval system:
1. Indexing: documents/items are first converted to vectors using deep learning models (aka embedding models). They are then stored in RAM and exportable to disk.
2. Retrieving: a user query is first converted to its vector representation. Retrieving modules then uses this query vector to evaluate similarity against indexed documents and returns top-scored ones.

In [ ]:
!pip install faiss
!pip install nmslib
!apt-get install libomp-dev

In [ ]:
#export
import faiss
import nmslib

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2022-01-28 05:20:34--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  29.6MB/s    in 0.2s    

2022-01-28 05:20:35 (29.6 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
import pandas as pd
data = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python', header=None).drop_duplicates()
data.columns = ['movieId', 'title', 'genres']
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
import tensorflow_hub as hub

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

We will use USE (Universal Sentence Encoder) model which has been trained to learn representation of a sentence semantic meaning from large public corpus.

In [ ]:
documents = data['title'].to_list()[:2000]
# # OOM for large document size
embeddings = embed(documents).numpy()
embeddings.shape

(2000, 512)

## Locally-Sensitive Hashing (LSH)
LSH is a very classical binary hash. Its core is to create multiple hash functions to map vectors into binary codes. Vectors closely related are expected to hashed into the same codes.

In [ ]:
#export
class DemoIndexLSH():
  def __init__(self, dimension, documents, embeddings):
    self.dimension = dimension
    self.documents = documents
    self.embeddings = embeddings

  def build(self, num_bits=8):
    self.index = faiss.IndexLSH(self.dimension, num_bits)
    self.index.add(self.embeddings)

  def query(self, input_embedding, k=5):
    distances, indices = self.index.search(input_embedding, k)

    return [(distance, self.documents[index]) for distance, index in zip(distances[0], indices[0])]

In [ ]:
index_lsh = DemoIndexLSH(512, documents, embeddings)
index_lsh.build(num_bits=16)

## Product Quantization with Inverted File (IVFPQ)
Product Quantization adopts k-means as its core quantizer and drastically increases the number of centroids by dividing each vector into many subvectors and run the quantizer on all of these subvectors.

In [ ]:
#export
class DemoIndexIVFPQ():
  def __init__(self, dimension, documents, embeddings):
    self.dimension = dimension
    self.documents = documents
    self.embeddings = embeddings

  def build(self,
            number_of_partition=2,
            number_of_subquantizers=2,
            subvector_bits=4):
    quantizer = faiss.IndexFlatL2(self.dimension)
    self.index = faiss.IndexIVFPQ(quantizer, 
                                  self.dimension,
                                  number_of_partition,
                                  number_of_subquantizers,
                                  subvector_bits)
    self.index.train(self.embeddings)
    self.index.add(self.embeddings)

  def query(self, input_embedding, k=5):
    distances, indices = self.index.search(input_embedding, k)

    return [(distance, self.documents[index]) for distance, index in zip(distances[0], indices[0])]

In [ ]:
#export
index_pq = DemoIndexIVFPQ(512, documents, embeddings)
index_pq.build()

## Hierarchical Navigable Small World Graphs (HNSW)
This method relies on exploring the graph based on closeness relation between a node and its neighbors and neighbor's neighbor and so on. HNSW stores the full length vectors and the full graph structure in memory (RAM).

In [ ]:
#export
class DemoHNSW():
  def __init__(self, dimension, documents, embeddings):
    self.dimension = dimension
    self.documents = documents
    self.embeddings = embeddings

  def build(self, num_bits=8):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.embeddings)
    self.index.createIndex({'post': 2}, print_progress=True)

  def query(self, input_embedding, k=5):
    indices, distances = self.index.knnQuery(input_embedding, k)

    return [(distance, self.documents[index]) for distance, index in zip(distances, indices)]

In [ ]:
index_hnsw = DemoHNSW(512, documents, embeddings)
index_hnsw.build()

In [ ]:
#export
class DemoIndexFlatL2():
  def __init__(self, dimension, documents, embeddings):
    self.dimension = dimension
    self.documents = documents
    self.embeddings = embeddings

  def build(self, num_bits=8):
    self.index = faiss.IndexFlatL2(self.dimension)
    self.index.add(self.embeddings)

  def query(self, input_embedding, k=5):
    distances, indices = self.index.search(input_embedding, k)

    return [(distance, self.documents[index]) for distance, index in zip(distances[0], indices[0])]

In [ ]:
index_flat = DemoIndexFlatL2(512, documents, embeddings)
index_flat.build()

In [ ]:
#export
def get_ann_top_items(embedding_model, ann_index, query, k=10):
    from timeit import default_timer as timer
    query_vector = embedding_model([query]).numpy()
    search_start = timer()
    top_docs = ann_index.query(query_vector, k)
    search_time = timer() - search_start
    print("search time: {:.2f} ms".format(search_time * 1000))
    return top_docs

In [ ]:
get_ann_top_items(embed, index_flat, "romance")

search time: 0.98 ms


[(0.9557337, 'True Romance (1993)'),
 (1.2160164, 'Love Serenade (1996)'),
 (1.2626684, 'Love Affair (1994)'),
 (1.3447756, 'Kissed (1996)'),
 (1.3752131, 'In Love and War (1996)'),
 (1.380403, 'Casablanca (1942)'),
 (1.3832322, 'Flirt (1995)'),
 (1.3862598, 'Moonlight and Valentino (1995)'),
 (1.3862815, 'Hotel de Love (1996)'),
 (1.3907105, 'Intimate Relations (1996)')]

In [ ]:
get_ann_top_items(embed, index_lsh, "romance")

search time: 0.48 ms


[(2.0, 'Visitors, The (Les Visiteurs) (1993)'),
 (2.0, 'City Hall (1996)'),
 (2.0, 'Paradise Road (1997)'),
 (3.0, 'When a Man Loves a Woman (1994)'),
 (3.0, 'Cosi (1996)'),
 (3.0, 'Haunted World of Edward D. Wood Jr., The (1995)'),
 (3.0, 'Eddie (1996)'),
 (3.0, 'Ransom (1996)'),
 (3.0, 'Time to Kill, A (1996)'),
 (3.0, 'Mirage (1995)')]

In [ ]:
get_ann_top_items(embed, index_pq, "romance")

search time: 0.45 ms


[(1.0426195, 'Falling in Love Again (1980)'),
 (1.0426195, 'Prom Night III: The Last Kiss (1989)'),
 (1.063688, 'Love in Bloom (1935)'),
 (1.063688, 'So Dear to My Heart (1949)'),
 (1.0789009, "Romy and Michele's High School Reunion (1997)"),
 (1.0789009, 'Farewell My Concubine (1993)'),
 (1.0789009, 'When Harry Met Sally... (1989)'),
 (1.0789009, 'Sex, Lies, and Videotape (1989)'),
 (1.0952816, 'Gaslight (1944)'),
 (1.0952816, 'Philadelphia Story, The (1940)')]

In [ ]:
get_ann_top_items(embed, index_hnsw, "romance")

search time: 0.47 ms


[(0.47786677, 'True Romance (1993)'),
 (0.60800815, 'Love Serenade (1996)'),
 (0.6313339, 'Love Affair (1994)'),
 (0.67238766, 'Kissed (1996)'),
 (0.68760645, 'In Love and War (1996)'),
 (0.6916159, 'Flirt (1995)'),
 (0.6931299, 'Moonlight and Valentino (1995)'),
 (0.6931407, 'Hotel de Love (1996)'),
 (0.6953552, 'Intimate Relations (1996)'),
 (0.69853836, 'Love in Bloom (1935)')]

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2022-01-28 05:39:47

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

nmslib        : 2.1.1
IPython       : 5.5.0
tensorflow_hub: 0.12.0
pandas        : 1.1.5
faiss         : 1.5.3
json          : 2.0.9

